<a href="https://colab.research.google.com/github/AtharvaTambat/WnCC-SoC-2022-QML/blob/main/Learning_Decomposition_of_a_Unitary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. The Code for Learning of a Unitary Operator 💻**

In [102]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib_inline
import math

# Initializing the device
dev1 = qml.device("default.qubit", wires=3)
U_prime = [[np.cos(math.pi/4),np.sin(math.pi/4)],[np.sin(math.pi/4),-np.cos(math.pi/4)]] # The target operator that we are trying to optimize/ learn to

# Let the Unitary gate which we have to learn be the Hadamard Gate
# Since any single-qubit gate can be represented by R_z(b)R_y(c)R_z(d), where b,c,d are angles (radians)
# NOTE: The global phase has been ignored

# The Quantum Circuit
@qml.qnode(dev1)
def circuit(params):
    
    # The parameterized Unitary Operator which we are trying to optimize - decomposed into three rotations
    qml.RZ(params[2], wires=1)
    qml.RY(params[1], wires=1)
    qml.RZ(params[0], wires=1)
    qml.Barrier()

    # Applying the H gate to the second qubit - which is the gate we have to learn our Unitary gate to
    qml.Hadamard(2) 
    qml.Barrier()
    
    # Applying the QSWAP circuit which will estimate the inner product of our parameterized U operator and the 
    # final operator U' (in this case, the H-gate), we have to learn to
    qml.Hadamard(0)
    qml.Toffoli(wires =[0,1,2])
    qml.Toffoli(wires =[0,2,1])
    qml.Toffoli(wires =[0,1,2])
    qml.Hadamard(0)
    qml.Barrier()

    return qml.probs(wires=[0])

# Defining the cost function for Optimization
def cost(x):
    probability = circuit(x)
    return -abs(probability[0]-probability[1])  # This is the cost function: -|<psi|phi>|^2 - where, |psi> = U|0> and, |phi> = U'|0>
# NOTE: Refer to the QSWAP .ipynb file for more details on the inner product formula in terms of probabilities of obtaining |0> and |1>

# Initializing the parameters
init_params = np.array([0.0001, 0.0002, 0.0001], requires_grad=True) # Contains the values of parameters b, c, d - which are to be optimized
print("Value of cost function with initial parameters (b, c, d):",cost(init_params),"\n\n\n")

# Drawing the circuit for initial parameters
print("The circuit with initial parameters (b, c, d)")
print(qml.draw(circuit)(init_params))
print("\n\n\n")

# Using Gradient Descent Algorithm 
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# Set the number of steps
steps = 100
# Set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Step: ",i+1,"   Cost:",cost(params))

print("\n\n\nThe values of b, c, d after variational learning are", params[0],",",params[1],",",params[2])




Value of cost function with initial parameters (b, c, d): -0.5000999999988331 



The circuit with initial parameters (b, c, d)
0: ────────────────────────────────||─────||──H─╭●─╭●─╭●──H──||─┤  Probs
1: ──RZ(0.00)──RY(0.00)──RZ(0.00)──||─────||────├●─├X─├●─────||─┤       
2: ────────────────────────────────||──H──||────╰X─╰●─╰X─────||─┤       




Step:  5    Cost: -0.889962635967772
Step:  10    Cost: -0.9865519909960416
Step:  15    Cost: -0.998533918627285
Step:  20    Cost: -0.9998423209276331
Step:  25    Cost: -0.9999830663372992
Step:  30    Cost: -0.9999981817272026
Step:  35    Cost: -0.9999998047640452
Step:  40    Cost: -0.9999999790366939
Step:  45    Cost: -0.9999999977490817
Step:  50    Cost: -0.9999999997583093
Step:  55    Cost: -0.9999999999740483
Step:  60    Cost: -0.9999999999972131
Step:  65    Cost: -0.9999999999997005
Step:  70    Cost: -0.9999999999999672
Step:  75    Cost: -0.999999999999996
Step:  80    Cost: -0.9999999999999991
Step:  85    Cost: -0.9999999

In [103]:
# Extracting parameters into variables
b = params[0]
c = params[1]
d = params[2]

# Initializing the rotation matrices based on the optimized values of parameters b, c, d
R_zb = [[complex(np.cos(b/2),-np.sin(b/2)),0],[0,complex(np.cos(b/2),np.sin(b/2))]]
R_yc = [[np.cos(c/2),-np.sin(c/2)],[np.sin(c/2),np.cos(c/2)]]
R_zd =[[complex(np.cos(d/2),-np.sin(d/2)),0],[0,complex(np.cos(d/2),np.sin(d/2))]]
op =   [[0,0],[0,0]]

# Calculating U = R_z(b)R_y(c)R_z(d), ignoring global phase
op = np.dot(R_zb,np.dot(R_yc,R_zd))
op2 = np.dot(op,[[1,0],[0,-1]]) # Second possibility of U

print("The possible gates are:\n")
for result in op:
  print(result)

print("\n")

for result in op2:
  print(result)


The possible gates are:

[ 0.70710678-3.53553391e-05j -0.70710678-3.53553390e-05j]
[0.70710678-3.53553390e-05j 0.70710678+3.53553391e-05j]


[0.70710678-3.53553391e-05j 0.70710678+3.53553390e-05j]
[ 0.70710678-3.53553390e-05j -0.70710678-3.53553391e-05j]


**NOTE:** Second possibility arises because for U$|0\rangle$, - U$|0\rangle$ also gives the lowest cost function.

1. U$|0\rangle$ is the second column of the 2 X 2 matrix representing the Unitary operator (Similarly for U').
2. Both U$|0\rangle$ and -U$|0\rangle$ give the same dot product with U'$|0\rangle$ (where, U' is the Target Operator, we intend to optimize to).
3. The cost function used in the above approach was -$\langle\psi|\phi\rangle^2$ (where $|\psi\rangle$ = U$|0\rangle$, and, $|\phi\rangle$ = U'$|0\rangle$)

In [107]:
# Calculate the error (= sum (a_{ij} - b_{ij})) in the two matrices obtained and the Target matrix
op_col2 = np.dot(op,[[0],[1]])
error1 = abs(op_col2[0] - U_prime[0][1]) + abs(op_col2[1] - U_prime[1][1]) 

op2_col2 = np.dot(op2,[[0],[1]])
error2 = abs(op2_col2[0] - U_prime[0][1]) + abs(op2_col2[1] - U_prime[1][1]) 

# Print that matrix whih has less error
if error1<error2:
  for result in op:
    print(result)
  ans = op
else:
  for result in op2:
    print(result)
  ans = op2



[0.70710678-3.53553391e-05j 0.70710678+3.53553390e-05j]
[ 0.70710678-3.53553390e-05j -0.70710678-3.53553391e-05j]


.......which is pretty good approximation of the H-gate 😸

# **2. No. of Qubits Used**
As can be seen from the above circuit, 3 qubits have been used

# **3. No. of Parameters to Learn**
Clearly, we have to learn 3 parameters.

# **4. The Loss (Cost) Function**
The cost function used in the above approach was $-\langle\psi|\phi\rangle^2$ (where $|\psi\rangle$ = U$|0\rangle$, and, $|\phi\rangle$ = U'$|0\rangle$).

# **5.a. Computed Matrix for the Operator**


In [109]:
# Print that matrix whih has less error
for result in ans:
    print(result)

[0.70710678-3.53553391e-05j 0.70710678+3.53553390e-05j]
[ 0.70710678-3.53553390e-05j -0.70710678-3.53553391e-05j]


# **5.b. Value of the Distance between the Learning and the Target Unitary**

In [119]:
sum = 0

for i in range(2):
  for j in range(2):
    sum += (ans[i][j] - U_prime[i][j])*((ans[i][j] - U_prime[i][j]).conjugate())

sum = math.sqrt(sum.real)
print("The value of the Distance between the Learning and the Target Unitary is:", sum)

The value of the Distance between the Learning and the Target Unitary is: 7.07106781119853e-05
